# 1小时入门SparkSQL

### 一，RDD，DataFrame和DataSet对比


DataFrame参照了Pandas的思想，在RDD基础上增加了schma，能够获取列名信息。

DataSet在DataFrame基础上进一步增加了数据类型信息，可以在编译时发现类型错误。

DataFrame可以看成DataSet[Row]，两者的API接口完全相同。

DataFrame和DataSet都支持SQL交互式查询，可以和 Hive无缝衔接。

DataSet只有Scala语言和Java语言接口中才支持，在Python和R语言接口只支持DataFrame。

![](RDD,DataFrame,Dataset对比.jpg)

In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
.builder()
.appName("Spark SQL basic example")
.enableHiveSupport()
.getOrCreate()

//以支持将RDD隐式转换成DataFrame
import spark.implicits._


### 二，创建DataFrame

1，通过toDF方法转换成DataFrame

可以将Seq,List或者 RDD转换成DataFrame


In [ ]:
//将Seq转换成DataFrame
val seq = Seq(
(1, "First Value", java.sql.Date.valueOf("2010-01-01")),
(2, "Second Value", java.sql.Date.valueOf("2010-02-01"))
)
val df = seq.toDF("int_column","string_column","date_column")
df.show

In [ ]:
//将List转换成DataFrame
val list = List(
("LiLei",15,88),
("HanMeiMei",16,90),
("DaChui",17,60)
)

val df = list.toDF("name","age","score")
df.show()
df.printSchema()


In [ ]:
//将RDD转换成DataFrame
val rdd = sc.parallelize(List(("LiLei",15),("HanMeiMei",17),("DaChui",16)),2)
val df = rdd.toDF("name","age")
df.show 


2, 通过CreateDataFrame方法动态创建DataFrame

可以通过createDataFrame的方法指定rdd和schema创建DataFrame。

这种方法比较繁琐，但是可以在预先不知道schema和数据类型的情况下在代码中动态创建DataFrame.


In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row

val schema = StructType(List(
StructField("integer_column", IntegerType, nullable = false),
StructField("string_column", StringType, nullable = true),
StructField("date_column", DateType, nullable = true)
))

val rdd = spark.sparkContext.parallelize(Seq(
Row(1, "First Value", java.sql.Date.valueOf("2010-01-01")),
Row(2, "Second Value", java.sql.Date.valueOf("2010-02-01")),
Row(2, "Second Value", null)
))
val df = spark.createDataFrame(rdd, schema)
df.show()


**3，通过读取文件创建**

可以读取json文件，csv文件，hive数据表或者mysql数据表得到DataFrame。

In [ ]:
//读取json文件生成DataFrame
val df = spark.read.json("resources/people.json")
df.show()

In [ ]:
//读取csv文件
val df = spark.sqlContext.read.format("com.databricks.spark.csv")
 .option("header","true") //如果在csv第一行有属性的话设置"true"，没有就是"false"
 .option("inferSchema","true")//这是自动推断属性列的数据类型。
 .option("delimiter", ",") //分隔符，默认为逗号 
 .load("resources/iris.csv")
df.show(5)
df.printSchema()

In [ ]:
//读取parquet文件
val df = spark.read.parquet("resources/users.parquet")
df.show

In [ ]:
//读取mysql数据表生成DataFrame
val url = "jdbc:mysql://localhost:3306/test"
 val df = spark.read
 .format("jdbc")
 .option("url", url)
 .option("dbtable", "runoob_tbl")
 .option("user", "root")
 .option("password", "0845")
 .load()
 df.show()


In [ ]:
//读取hive数据表生成DataFrame
import java.io.File
import org.apache.spark.sql.{Row, SaveMode, SparkSession}

val warehouseLocation = new File("spark-warehouse").getAbsolutePath

val spark = SparkSession
  .builder()
  .appName("Spark Hive Example")
  .config("spark.sql.warehouse.dir", warehouseLocation)
  .enableHiveSupport()
  .getOrCreate()

import spark.implicits._
import spark.sql

sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
sql("LOAD DATA LOCAL INPATH 'resources/kv1.txt' INTO TABLE src")

val df = sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")

### 三，创建DataSet


DataSet主要通过toDS方法从Seq,List或者RDD数据类型转换得到，或者从DataFrame通过as方法转换得到。

1，通过toDS方法转换得到DataSet

In [ ]:
//将Seq转换成DataSet
import spark.implicits._
case class Student(name:String,age:Int)

val seq = Seq(
Student("LiLei",16),
Student("DaChui",17),
Student("HanMeiMei",15)
)

val ds = seq.toDS
ds.show

In [ ]:
//将RDD转换成DataSet
import spark.implicits._
case class Student(name:String,age:Int)

val rdd = sc.parallelize(List(Student("LiLei",15),
                              Student("HanMeiMei",17),
                              Student("DaChui",16)))
val ds = rdd.toDS
ds.show 

2，通过DataFrame的as转换方法得到DataSet


In [ ]:
//读取json文件并转换生成DataSet
import spark.implicits._
case class People(age:Long,name:String)
val ds = spark.read.json("resources/people.json").as[People]
ds.show()

In [ ]:
//读取csv文件并转换得到DataSet
import spark.implicits._
case class Flower(sepallength:Double,sepalwidth:Double,
                  petallength:Double,petalwidth:Double,label:Int)
val ds = spark.sqlContext.read.format("com.databricks.spark.csv")
 .option("header","true") //如果在csv第一行有属性的话设置true，没有就是"false"
 .option("inferSchema","true")//这是自动推断属性列的数据类型。
 .option("delimiter", ",") //分隔符，默认为逗号 
 .load("resources/iris.csv")
 .as[Flower]
ds.show()
ds.printSchema()

In [ ]:
ds.getClass.getSimpleName

### 四，DataFrame/DataSet保存成文件

可以保存成csv文件，json文件，parquet文件或者保存成hive数据表

In [40]:
//保存成csv文件
val peopleDF = spark.read.format("json").load("resources/people.json")
peopleDF.write.format("csv").option("header","true").save("people.csv")

peopleDF = [age: bigint, name: string]


[age: bigint, name: string]

In [41]:
//先转换成rdd再保存成txt文件
peopleDF.rdd.saveAsTextFile("newpeople.txt")

In [42]:
//保存成json文件
peopleDF.write.json("people.json")

In [43]:
// 保存成parquet 文件, 可以存储schema信息
peopleDF.write.partitionBy("age").format("parquet").save("namesAndAges.parquet")
peopleDF.write.parquet("people.parquet")

In [ ]:
// 保存成hive数据表
val peopleDF = spark.read.format("json").load("resources/people.json")
peopleDF.write.bucketBy(42, "name").sortBy("age").saveAsTable("people_bucketed")

### 五，DataFrame的API交互

In [44]:
import spark.implicits._

val list = List(
("LiLei",15,"male"),
("HanMeiMei",16,"female"),
("DaChui",17,"male")
)

val df = list.toDF("name","age","gender")
df.show()
df.printSchema()


+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
+---------+---+------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- gender: string (nullable = true)



list = List((LiLei,15,male), (HanMeiMei,16,female), (DaChui,17,male))
df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

**1，Action操作**

DataFrame的Action操作包括show,count,collect,collectAsList,describe,take,takeAsList,head,first等操作。

In [45]:
//show
df.show()

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
+---------+---+------+



In [46]:
//show(numRows: Int, truncate: Boolean) 
//第二个参数设置是否当输出字段长度超过20时进行截取
df.show(2,false) 

+---------+---+------+
|name     |age|gender|
+---------+---+------+
|LiLei    |15 |male  |
|HanMeiMei|16 |female|
+---------+---+------+
only showing top 2 rows



In [48]:
//count
df.count

3

In [49]:
//collect
df.collect()

Array([LiLei,15,male], [HanMeiMei,16,female], [DaChui,17,male])

In [50]:
//collectAsList
df.collectAsList()

[[LiLei,15,male], [HanMeiMei,16,female], [DaChui,17,male]]

In [51]:
//first
df.first

[LiLei,15,male]

In [52]:
//take
df.take(3)

Array([LiLei,15,male], [HanMeiMei,16,female], [DaChui,17,male])

In [53]:
//head
df.head(2)

Array([LiLei,15,male], [HanMeiMei,16,female])

In [54]:
//takeAsList
df.takeAsList(2)

[[LiLei,15,male], [HanMeiMei,16,female]]

**2，类RDD操作** 

DataFrame支持RDD常用的map,flatMap,filter,reduce,distinct,

cache,sample,mapPartitions,foreach,intersect,except等操作。

可以把DataFrame当做数据类型为Row的RDD来进行操作。

In [55]:
import spark.implicits._
val df = List("Hello World","Hello China","Hello Spark").toDF("value")
df.show

+-----------+
|      value|
+-----------+
|Hello World|
|Hello China|
|Hello Spark|
+-----------+



df = [value: string]


[value: string]

In [56]:
//map
df.map(x=>x(0).toString.toUpperCase).show

+-----------+
|      value|
+-----------+
|HELLO WORLD|
|HELLO CHINA|
|HELLO SPARK|
+-----------+



In [57]:
//flatMap
val flatdf = df.flatMap(x=>x(0).toString.split(" "))
flatdf.show

+-----+
|value|
+-----+
|Hello|
|World|
|Hello|
|China|
|Hello|
|Spark|
+-----+



flatdf = [value: string]


[value: string]

In [58]:
//distinct
flatdf.distinct.show

+-----+
|value|
+-----+
|World|
|China|
|Hello|
|Spark|
+-----+



In [59]:
//cache缓存到内存，checkpoint永久性保存到磁盘 
df.cache

[value: string]

In [60]:
//filter过滤
df.filter(s=>s(0).toString.endsWith("Spark")).show

+-----------+
|      value|
+-----------+
|Hello Spark|
+-----------+



In [61]:
//sample抽样
df.sample(false,0.6,0).show

+-----------+
|      value|
+-----------+
|Hello China|
|Hello Spark|
+-----------+



In [62]:
val df2 = Seq("Hello World","Hello Scala","Hello Spark").toDF("value")
df2.show

+-----------+
|      value|
+-----------+
|Hello World|
|Hello Scala|
|Hello Spark|
+-----------+



df2 = [value: string]


[value: string]

In [63]:
//intersect交集
df.intersect(df2).show

+-----------+
|      value|
+-----------+
|Hello Spark|
|Hello World|
+-----------+



In [64]:
//except补集
df.except(df2).show

+-----------+
|      value|
+-----------+
|Hello China|
+-----------+



**3，类Excel操作**

可以对DataFrame进行增加列，删除列，重命名列，排序等操作，去除重复行，去除空行，就跟操作Excel表格一样。

In [65]:
import spark.implicits._

val list = List(
("LiLei",15,"male"),
("HanMeiMei",16,"female"),
("DaChui",17,"male"),
("RuHua",16,null)
)

val df = list.toDF("name","age","gender")
df.show()
df.printSchema()

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
|    RuHua| 16|  null|
+---------+---+------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- gender: string (nullable = true)



list = List((LiLei,15,male), (HanMeiMei,16,female), (DaChui,17,male), (RuHua,16,null))
df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [66]:
//增加列
df.withColumn("birthyear",-df("age")+2019).show

+---------+---+------+---------+
|     name|age|gender|birthyear|
+---------+---+------+---------+
|    LiLei| 15|  male|     2004|
|HanMeiMei| 16|female|     2003|
|   DaChui| 17|  male|     2002|
|    RuHua| 16|  null|     2003|
+---------+---+------+---------+



In [67]:
//增加序号列
import org.apache.spark.sql.functions.monotonically_increasing_id
val dfnew = df.withColumn("idx", monotonically_increasing_id)
dfnew.show

+---------+---+------+---+
|     name|age|gender|idx|
+---------+---+------+---+
|    LiLei| 15|  male|  0|
|HanMeiMei| 16|female|  1|
|   DaChui| 17|  male|  2|
|    RuHua| 16|  null|  3|
+---------+---+------+---+



dfnew = [name: string, age: int ... 2 more fields]


[name: string, age: int ... 2 more fields]

In [68]:
//置换列的顺序
val df = dfnew.select("idx","name","age","gender")
df.show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|    RuHua| 16|  null|
+---+---------+---+------+



df = [idx: bigint, name: string ... 2 more fields]


[idx: bigint, name: string ... 2 more fields]

In [69]:
//删除列
df.drop("gender").show

+---+---------+---+
|idx|     name|age|
+---+---------+---+
|  0|    LiLei| 15|
|  1|HanMeiMei| 16|
|  2|   DaChui| 17|
|  3|    RuHua| 16|
+---+---------+---+



In [70]:
//重命名列
df.withColumnRenamed("gender","sex").show

+---+---------+---+------+
|idx|     name|age|   sex|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|    RuHua| 16|  null|
+---+---------+---+------+



In [71]:
//排序sort，可以指定升序降序
df.sort($"age".desc).show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  2|   DaChui| 17|  male|
|  1|HanMeiMei| 16|female|
|  3|    RuHua| 16|  null|
|  0|    LiLei| 15|  male|
+---+---------+---+------+



In [72]:
//排序orderby,默认为升序,可以根据多个字段
df.orderBy(df("age").desc,df("gender").desc).show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  2|   DaChui| 17|  male|
|  1|HanMeiMei| 16|female|
|  3|    RuHua| 16|  null|
|  0|    LiLei| 15|  male|
+---+---------+---+------+



In [73]:
//去除nan值行
df.na.drop.show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
+---+---------+---+------+



In [74]:
//填充nan值
df.na.fill("female").show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|    RuHua| 16|female|
+---+---------+---+------+



In [75]:
//替换某些值
df.na.replace(Seq("gender","name"),Map(" "->"female","RuHua"->"SiYu")).show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|     SiYu| 16|  null|
+---+---------+---+------+



In [76]:
//去重，默认根据全部字段
val df2 = df.unionAll(df)
df2.show
df2.dropDuplicates().show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|    RuHua| 16|  null|
|  0|    LiLei| 15|  male|
|  1|HanMeiMei| 16|female|
|  2|   DaChui| 17|  male|
|  3|    RuHua| 16|  null|
+---+---------+---+------+

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  1|HanMeiMei| 16|female|
|  0|    LiLei| 15|  male|
|  3|    RuHua| 16|  null|
|  2|   DaChui| 17|  male|
+---+---------+---+------+



df2 = [idx: bigint, name: string ... 2 more fields]


[idx: bigint, name: string ... 2 more fields]

In [77]:
//去重,根据部分字段
df.dropDuplicates("age").show

+---+---------+---+------+
|idx|     name|age|gender|
+---+---------+---+------+
|  1|HanMeiMei| 16|female|
|  0|    LiLei| 15|  male|
|  2|   DaChui| 17|  male|
+---+---------+---+------+



In [78]:
//简单聚合操作
df.agg("name"->"count","age"->"max").show

+-----------+--------+
|count(name)|max(age)|
+-----------+--------+
|          4|      17|
+-----------+--------+



In [79]:
//汇总信息
df.describe().show

+-------+------------------+------+-----------------+------+
|summary|               idx|  name|              age|gender|
+-------+------------------+------+-----------------+------+
|  count|                 4|     4|                4|     3|
|   mean|               1.5|  null|             16.0|  null|
| stddev|1.2909944487358056|  null|0.816496580927726|  null|
|    min|                 0|DaChui|               15|female|
|    max|                 3| RuHua|               17|  male|
+-------+------------------+------+-----------------+------+



In [80]:
//频率超过0.5的年龄和性别
df.stat.freqItems(Seq("age","gender"),0.5).show

+-------------+----------------+
|age_freqItems|gender_freqItems|
+-------------+----------------+
|         [16]|          [male]|
+-------------+----------------+



**4，类SQL表操作**

类sql表操作包括表查询(select,selectExpr,where,filter),表连接(join,union,unionAll),表分组(groupby,agg,pivot)等操作。

In [81]:
import spark.implicits._

val list = List(
("LiLei",15,"male"),
("HanMeiMei",16,"female"),
("DaChui",17,"male"),
("RuHua",16,null)
)

val df = list.toDF("name","age","gender")
df.show()

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
|    RuHua| 16|  null|
+---------+---+------+



list = List((LiLei,15,male), (HanMeiMei,16,female), (DaChui,17,male), (RuHua,16,null))
df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [82]:
//表查询select
df.select($"name").limit(2).show

+---------+
|     name|
+---------+
|    LiLei|
|HanMeiMei|
+---------+



In [83]:
df.select($"name",$"age" + 1).show

+---------+---------+
|     name|(age + 1)|
+---------+---------+
|    LiLei|       16|
|HanMeiMei|       17|
|   DaChui|       18|
|    RuHua|       17|
+---------+---------+



In [84]:
//表查询select
df.select($"name",-$"age"+2019).toDF("name","birth_year").show

+---------+----------+
|     name|birth_year|
+---------+----------+
|    LiLei|      2004|
|HanMeiMei|      2003|
|   DaChui|      2002|
|    RuHua|      2003|
+---------+----------+



In [85]:
//表查询selectExpr,可以使用UDF函数，指定别名等
df.selectExpr("name", "2019-age as birth_year" , "UPPER(gender) as gender" ).show

+---------+----------+------+
|     name|birth_year|gender|
+---------+----------+------+
|    LiLei|      2004|  MALE|
|HanMeiMei|      2003|FEMALE|
|   DaChui|      2002|  MALE|
|    RuHua|      2003|  null|
+---------+----------+------+



In [86]:
//表查询where, 指定SQL中的where字句表达式
df.where("gender='male' and age>15").show

+------+---+------+
|  name|age|gender|
+------+---+------+
|DaChui| 17|  male|
+------+---+------+



In [87]:
//表查询filter
df.filter($"age">16).show

+------+---+------+
|  name|age|gender|
+------+---+------+
|DaChui| 17|  male|
+------+---+------+



In [88]:
//表查询filter,注意不等号的写法
df.filter($"gender"=!="male").show

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|HanMeiMei| 16|female|
+---------+---+------+



In [89]:
//表查询filter,注意等于号的写法
df.filter($"gender"==="male").show

+------+---+------+
|  name|age|gender|
+------+---+------+
| LiLei| 15|  male|
|DaChui| 17|  male|
+------+---+------+



In [90]:
//表查询filter
df.filter("gender ='male'").show

+------+---+------+
|  name|age|gender|
+------+---+------+
| LiLei| 15|  male|
|DaChui| 17|  male|
+------+---+------+



In [91]:
//表连接join
val dfscore = Seq(("LiLei","male",88),("HanMeiMei","female",90),("DaChui","male",50))
              .toDF("name","gender","score")

dfscore.show

+---------+------+-----+
|     name|gender|score|
+---------+------+-----+
|    LiLei|  male|   88|
|HanMeiMei|female|   90|
|   DaChui|  male|   50|
+---------+------+-----+



dfscore = [name: string, gender: string ... 1 more field]


[name: string, gender: string ... 1 more field]

In [92]:
//表连接join,根据单个字段
df.join(dfscore.select("name","score"),"name").show

+---------+---+------+-----+
|     name|age|gender|score|
+---------+---+------+-----+
|    LiLei| 15|  male|   88|
|HanMeiMei| 16|female|   90|
|   DaChui| 17|  male|   50|
+---------+---+------+-----+



In [93]:
//表连接join,根据多个字段
df.join(dfscore,Seq("name","gender")).show

+---------+------+---+-----+
|     name|gender|age|score|
+---------+------+---+-----+
|    LiLei|  male| 15|   88|
|HanMeiMei|female| 16|   90|
|   DaChui|  male| 17|   50|
+---------+------+---+-----+



In [94]:
//表连接join,根据多个字段
//可以指定连接方式为"inner","left","right","outer"
df.join(dfscore,Seq("name","gender"),"right").show

+---------+------+---+-----+
|     name|gender|age|score|
+---------+------+---+-----+
|    LiLei|  male| 15|   88|
|HanMeiMei|female| 16|   90|
|   DaChui|  male| 17|   50|
+---------+------+---+-----+



In [95]:
df.join(dfscore,Seq("name","gender"),"outer").show

+---------+------+---+-----+
|     name|gender|age|score|
+---------+------+---+-----+
|HanMeiMei|female| 16|   90|
|   DaChui|  male| 17|   50|
|    LiLei|  male| 15|   88|
|    RuHua|  null| 16| null|
+---------+------+---+-----+



In [96]:
//表连接，灵活指定连接关系
val dfmark = dfscore.withColumnRenamed("gender","sex")
dfmark.show

+---------+------+-----+
|     name|   sex|score|
+---------+------+-----+
|    LiLei|  male|   88|
|HanMeiMei|female|   90|
|   DaChui|  male|   50|
+---------+------+-----+



dfmark = [name: string, sex: string ... 1 more field]


[name: string, sex: string ... 1 more field]

In [97]:
df.join(dfmark,df("name")===dfmark("name")&&df("gender")===dfmark("sex"),
        "inner").show

+---------+---+------+---------+------+-----+
|     name|age|gender|     name|   sex|score|
+---------+---+------+---------+------+-----+
|    LiLei| 15|  male|    LiLei|  male|   88|
|HanMeiMei| 16|female|HanMeiMei|female|   90|
|   DaChui| 17|  male|   DaChui|  male|   50|
+---------+---+------+---------+------+-----+



In [98]:
//表合并union,unionAll
val dfstudent = Seq(("Jim",18,"male"),("Lily",16,"female")).toDF("name","age","gender")
dfstudent.show

+----+---+------+
|name|age|gender|
+----+---+------+
| Jim| 18|  male|
|Lily| 16|female|
+----+---+------+



dfstudent = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [101]:
df.union(dfstudent).show

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
|    RuHua| 16|  null|
|      Jim| 18|  male|
|     Lily| 16|female|
+---------+---+------+



In [102]:
//表分组 groupBy
import org.apache.spark.sql.functions._
df.groupBy("gender").max("age").show

+------+--------+
|gender|max(age)|
+------+--------+
|  null|      16|
|female|      16|
|  male|      17|
+------+--------+



In [103]:
//表分组后聚合，groupBy,agg
df.groupBy("gender").agg(mean("age") as "mean_age",collect_list("name") as "names").show

+------+--------+---------------+
|gender|mean_age|          names|
+------+--------+---------------+
|  null|    16.0|        [RuHua]|
|female|    16.0|    [HanMeiMei]|
|  male|    16.0|[LiLei, DaChui]|
+------+--------+---------------+



In [104]:
//表分组聚合，groupBy,agg
df.groupBy("gender").agg("age"->"avg","name"->"collect_list","name"->"count").show

+------+--------+------------------+-----------+
|gender|avg(age)|collect_list(name)|count(name)|
+------+--------+------------------+-----------+
|  null|    16.0|           [RuHua]|          1|
|female|    16.0|       [HanMeiMei]|          1|
|  male|    16.0|   [LiLei, DaChui]|          2|
+------+--------+------------------+-----------+



In [105]:
//表分组聚合，groupBy,agg
df.groupBy("gender","age").agg("name"->"collect_list").show

+------+---+------------------+
|gender|age|collect_list(name)|
+------+---+------------------+
|  male| 17|          [DaChui]|
|  male| 15|           [LiLei]|
|  null| 16|           [RuHua]|
|female| 16|       [HanMeiMei]|
+------+---+------------------+



In [106]:
//表分组后透视，groupBy,pivot
val dfstudent = Seq(("LiLei",18,"male",1),("HanMeiMei",16,"female",1),
                    ("Jim",17,"male",2),("DaChui",20,"male",2))
                .toDF("name","age","gender","class")
dfstudent.show
dfstudent.groupBy("class").pivot("gender").max("age").show

+---------+---+------+-----+
|     name|age|gender|class|
+---------+---+------+-----+
|    LiLei| 18|  male|    1|
|HanMeiMei| 16|female|    1|
|      Jim| 17|  male|    2|
|   DaChui| 20|  male|    2|
+---------+---+------+-----+

+-----+------+----+
|class|female|male|
+-----+------+----+
|    1|    16|  18|
|    2|  null|  20|
+-----+------+----+



dfstudent = [name: string, age: int ... 2 more fields]


[name: string, age: int ... 2 more fields]

In [107]:
//explode行转列

val helloDF = Seq("hello world","hello China","hello Spark").toDF("value")
helloDF.show

+-----------+
|      value|
+-----------+
|hello world|
|hello China|
|hello Spark|
+-----------+



helloDF = [value: string]


[value: string]

In [108]:
helloDF.explode("value","parts"){s:String => s.split(" ")}.show

+-----------+-----+
|      value|parts|
+-----------+-----+
|hello world|hello|
|hello world|world|
|hello China|hello|
|hello China|China|
|hello Spark|hello|
|hello Spark|Spark|
+-----------+-----+



### 六，DataFrame的SQL交互

将DataFrame/DataSet注册为临时表视图或者全局表视图后，可以使用sql语句对DataFrame进行交互。

以下为示范代码。

In [ ]:
//注册为临时表视图
val df = Seq(("LiLei",18,"male"),("HanMeiMei",17,"female"),("Jim",16,"male")).
      toDF("name","age","gender")

df.createOrReplaceTempView("student")
val sqlDF = spark.sql("select * from student limit 2")
sqlDF.show()

In [ ]:
// 注册为全局临时表视图
df.createGlobalTempView("student")

spark.sql("SELECT * FROM global_temp.student").show()

//可以在新的Session中访问
spark.newSession().sql("SELECT * FROM global_temp.student").show()

In [ ]:
//与Hive交互操作的范例

import java.io.File
import org.apache.spark.sql.{Row, SaveMode, SparkSession}
case class Record(key: Int, value: String)
// warehouseLocation points to the default location for managed databases and tables
val warehouseLocation = new File("spark-warehouse").getAbsolutePath

val spark = SparkSession
  .builder()
  .appName("Spark Hive Example")
  .config("spark.sql.warehouse.dir", warehouseLocation)
  .enableHiveSupport()
  .getOrCreate()

import spark.implicits._
import spark.sql

sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
sql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")

// Queries are expressed in HiveQL
sql("SELECT * FROM src").show()

// Aggregation queries are also supported.
sql("SELECT COUNT(*) FROM src").show()

// The results of SQL queries are themselves DataFrames and support all normal functions.
val sqlDF = sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")

// The items in DataFrames are of type Row, you can access each column by ordinal.
val stringsDS = sqlDF.map {
  case Row(key: Int, value: String) => s"Key: $key, Value: $value"
}
stringsDS.show()

// You can also use DataFrames to create temporary views within a SparkSession.
val recordsDF = spark.createDataFrame((1 to 100).map(i => Record(i, s"val_$i")))
recordsDF.createOrReplaceTempView("records")

// Queries can then join DataFrame data with data stored in Hive.
sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

// Create a Hive managed Parquet table, with HQL syntax instead of the Spark SQL 
// `USING hive`
sql("CREATE TABLE hive_records(key int, value string) STORED AS PARQUET")
// Save DataFrame to the Hive managed table
val df = spark.table("src")
df.write.mode(SaveMode.Overwrite).saveAsTable("hive_records")
// After insertion, the Hive managed table has data now
sql("SELECT * FROM hive_records").show()

val dataDir = "/tmp/parquet_data"
spark.range(10).write.parquet(dataDir)
sql(s"CREATE EXTERNAL TABLE hive_ints(key int) STORED AS PARQUET LOCATION '$dataDir'")
sql("SELECT * FROM hive_ints").show()

// Turn on flag for Hive Dynamic Partitioning
spark.sqlContext.setConf("hive.exec.dynamic.partition", "true")
spark.sqlContext.setConf("hive.exec.dynamic.partition.mode", "nonstrict")

// Create a Hive partitioned table using DataFrame API
df.write.partitionBy("key").format("hive").saveAsTable("hive_part_tbl")
sql("SELECT * FROM hive_part_tbl").show()
spark.stop()


### 七，RDD，DataFrame和 DataSet的相互转换

三种数据结构RDD，DataFrame和DataSet之间可以相互转换。

In [ ]:
//RDD转DataFrame
import spark.implicits
val rdd = sc.parallelize(List(("LiLei",15),("HanMeiMei",17),("DaChui",16)),2)
val df = rdd.toDF("name","age")

In [ ]:
//RDD转换DataSet
import spark.implicits._
val rdd = sc.parallelize(List(("LiLei",15),("HanMeiMei",17),("DaChui",16)),2)
case class student(name:String, age:Int)
val ds = rdd.map(s => student(s._1,s._2)).toDS

In [ ]:
//DataFrame或DataSet转RDD
import spark.implicits._
val rdd1 = df.rdd
val rdd2 = ds.rdd

In [ ]:
//DataSet转DataFrame
import spark.implicits._
val studentDF = ds.toDF
studentDF.show

In [ ]:
//DataFrame转DataSet
import spark.implicits._
case class Student(name:String, age:Int)
val studentDS = df.as[Student]
studentDS.show

### 八，用户自定义函数

SparkSQL的用户自定义函数包括二种类型，UDF和UDAF，即普通用户自定义函数和用户自定义聚合函数。

其中UDAF由分为弱类型UDAF和强类型UDAF，前者可以在DataFrame和DataSet中使用，

后者仅能够在DataSet中使用。

**1，普通UDF**

In [109]:
import spark.implicits._

val list = List(
("LiLei",15,"male"),
("HanMeiMei",16,"female"),
("DaChui",17,"male"),
("RuHua",16,null)
)

val df = list.toDF("name","age","gender")
df.show()

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 15|  male|
|HanMeiMei| 16|female|
|   DaChui| 17|  male|
|    RuHua| 16|  null|
+---------+---+------+



list = List((LiLei,15,male), (HanMeiMei,16,female), (DaChui,17,male), (RuHua,16,null))
df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [110]:
spark.udf.register("addName",(x:String)=>"Name:"+x)

UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [112]:
df.selectExpr("addName(name) as name","age").show()

+--------------+---+
|          name|age|
+--------------+---+
|    Name:LiLei| 15|
|Name:HanMeiMei| 16|
|   Name:DaChui| 17|
|    Name:RuHua| 16|
+--------------+---+



**2, 弱类型UDAF**

弱类型UDAF需要继承UserDefinedAggregateFunction

In [137]:
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.types._
import org.apache.spark.sql.{Row, SparkSession}

object MyAverage extends UserDefinedAggregateFunction {
  //聚合函数输入的类型
  override def inputSchema: StructType = 
  StructType(StructField("inputColumn", LongType) :: Nil)

  //聚合函数缓冲区类型
  override def bufferSchema: StructType = 
  StructType(StructField("sum", LongType) :: StructField("column", LongType) :: Nil)

  //返回值类型
  override def dataType: DataType = DoubleType

  //相同输入是否返回相同输出
  override def deterministic: Boolean = true

  //初始化
  override def initialize(buffer: MutableAggregationBuffer): Unit = {
    buffer(0) = 0L
    buffer(1) = 0L
  }

  //相同数据合并
  override def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
    if (!input.isNullAt(0)) {
      buffer(0) = buffer.getLong(0) + input.getLong(0)
      buffer(1) = buffer.getLong(1) + 1
    }
  }

  //不同Executor之间的数据合并
  override def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
    buffer1(0) = buffer1.getLong(0) + buffer2.getLong(0)
    buffer1(1) = buffer1.getLong(1) + buffer2.getLong(1)
  }

  //计算结果
  override def evaluate(buffer: Row): Any = buffer.getLong(0).toDouble / buffer.getLong(1)
}

defined object MyAverage


In [138]:
spark.udf.register("MyAverage", MyAverage)

MyAverage$@339e5321

In [139]:
val df = Seq(("LiLei",18,"male"),("HanMeiMei",17,"female"),("Jim",16,"male")).
      toDF("name","age","gender")
df.show

+---------+---+------+
|     name|age|gender|
+---------+---+------+
|    LiLei| 18|  male|
|HanMeiMei| 17|female|
|      Jim| 16|  male|
+---------+---+------+



df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [140]:
df.agg("age"->"MyAverage").show

+--------------+
|myaverage(age)|
+--------------+
|          17.0|
+--------------+



In [141]:
import spark.implicits._
case class Student(name:String,age:Int,gender:String)
val ds = df.as[Student]
ds.agg("age"->"MyAverage").show

+--------------+
|myaverage(age)|
+--------------+
|          17.0|
+--------------+



defined class Student
ds = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

**3，强类型UDAF**

强类型UDAF需要继承自Aggregator，不可注册

In [125]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.{Encoder, Encoders, SparkSession}

case class Employee(name: String, salary: Long)
case class Average(var sum: Long, var count: Long)

object MyAverage2 extends Aggregator[Employee, Average, Double] {
  //定义一个数据结构，保存工资总数和工资总个数，初始都为0
  override def zero: Average = Average(0L, 0L)

  //统计数据
  override def reduce(b: Average, a: Employee): Average = {
    b.sum += a.salary
    b.count += 1
    b
  }

  //各个Executor数据汇总
  override def merge(b1: Average, b2: Average): Average = {
    b1.sum += b2.sum
    b1.count += b2.count
    b1
  }

  //计算输出
  override def finish(reduction: Average): Double 
  = reduction.sum.toDouble / reduction.count

  // 设定之间值类型的编码器，要转换成case类
  // Encoders.product是进行scala元组和case类转换的编码器
  override def bufferEncoder: Encoder[Average] = Encoders.product

  //设置最终输出编码器
  override def outputEncoder: Encoder[Double] = Encoders.scalaDouble
}


defined class Employee
defined class Average
defined object MyAverage2


In [128]:
import spark.implicits._

val ds = spark.read.json("resources/employees.json").as[Employee]
ds.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



ds = [name: string, salary: bigint]


[name: string, salary: bigint]

In [133]:
//将MyAverage2转换成TypedColumn并给它命名
val average_salary = MyAverage2.toColumn.name("average_salary")
ds.select(average_salary).show()

+--------------+
|average_salary|
+--------------+
|        3750.0|
+--------------+



average_salary = myaverage2() AS `average_salary`


lastException: Throwable = null


myaverage2() AS `average_salary`